In [2]:
import sys
print("Python executable:", sys.executable)

import langchain
import langgraph
import sqlalchemy
import pinecone
import sentence_transformers

print("All core imports successful ✅")

Python executable: c:\Users\GCV\dev\work\agentic-gst-ai\gstagent\Scripts\python.exe
All core imports successful ✅


In [1]:
from sqlalchemy import create_engine, Column, Integer, String, Float
from sqlalchemy.orm import declarative_base, sessionmaker
import os

In [2]:
# Create a local SQLite database file
DB_PATH = "invoices.db"
DATABASE_URL = f"sqlite:///{DB_PATH}"

engine = create_engine(DATABASE_URL, echo=False)
SessionLocal = sessionmaker(bind=engine)

print("SQLite engine created at:", os.path.abspath(DB_PATH))


SQLite engine created at: c:\Users\GCV\dev\work\agentic-gst-ai\notebooks\invoices.db


In [3]:
Base = declarative_base()

class Invoice(Base):
    __tablename__ = "invoices"

    id = Column(Integer, primary_key=True, index=True)
    invoice_date = Column(String, nullable=False)
    location = Column(String, nullable=False)
    vendor = Column(String, nullable=False)
    item = Column(String, nullable=False)
    quantity = Column(Integer, nullable=False)
    amount = Column(Float, nullable=False)
    gst_rate = Column(Float, nullable=False)

    def __repr__(self):
        return (
            f"<Invoice(id={self.id}, date={self.invoice_date}, "
            f"vendor={self.vendor}, item={self.item}, amount={self.amount})>"
        )


In [4]:
# Create tables in the database
Base.metadata.create_all(bind=engine)

print("Invoices table created successfully ✅")


Invoices table created successfully ✅


In [5]:
# Manual sanity check insert
session = SessionLocal()

test_invoice = Invoice(
    invoice_date="2025-01-05",
    location="Bangalore",
    vendor="Amazon",
    item="Laptop",
    quantity=1,
    amount=65000.0,
    gst_rate=18.0
)

session.add(test_invoice)
session.commit()

print("Test invoice inserted with ID:", test_invoice.id)
session.close()


Test invoice inserted with ID: 1


In [6]:
# Read back data
session = SessionLocal()
invoices = session.query(Invoice).all()

for inv in invoices:
    print(inv)

session.close()

<Invoice(id=1, date=2025-01-05, vendor=Amazon, item=Laptop, amount=65000.0)>


In [7]:
from sqlalchemy import text

In [8]:
def run_sql_tool(query: str) -> str:
    """
    Executes a raw SQL query against the invoices database.
    Supports SELECT and INSERT queries.
    
    Returns:
        - SELECT: stringified rows
        - INSERT: confirmation message
        - ERROR: error message
    """
    try:
        query = query.strip()
        session = SessionLocal()

        # Detect query type
        is_select = query.lower().startswith("select")

        if is_select:
            result = session.execute(text(query))
            rows = result.fetchall()

            if not rows:
                return "No records found."

            # Convert rows to readable format
            output = []
            for row in rows:
                output.append(dict(row._mapping))

            return str(output)

        else:
            # INSERT / UPDATE / DELETE
            session.execute(text(query))
            session.commit()
            return "Query executed successfully."

    except Exception as e:
        return f"SQL ERROR: {str(e)}"

    finally:
        session.close()


In [9]:
# Test SELECT via tool
query = "SELECT * FROM invoices;"
print(run_sql_tool(query))

[{'id': 1, 'invoice_date': '2025-01-05', 'location': 'Bangalore', 'vendor': 'Amazon', 'item': 'Laptop', 'quantity': 1, 'amount': 65000.0, 'gst_rate': 18.0}]


In [10]:
# Test INSERT via tool
insert_query = """
INSERT INTO invoices (invoice_date, location, vendor, item, quantity, amount, gst_rate)
VALUES ('2025-01-10', 'Chennai', 'Flipkart', 'Headphones', 2, 4000, 18);
"""

print(run_sql_tool(insert_query))


Query executed successfully.


In [11]:
print(run_sql_tool("SELECT * FROM invoices;"))

[{'id': 1, 'invoice_date': '2025-01-05', 'location': 'Bangalore', 'vendor': 'Amazon', 'item': 'Laptop', 'quantity': 1, 'amount': 65000.0, 'gst_rate': 18.0}, {'id': 2, 'invoice_date': '2025-01-10', 'location': 'Chennai', 'vendor': 'Flipkart', 'item': 'Headphones', 'quantity': 2, 'amount': 4000.0, 'gst_rate': 18.0}]


In [12]:
seed_queries = [
    """
    INSERT INTO invoices (invoice_date, location, vendor, item, quantity, amount, gst_rate)
    VALUES ('2025-01-05', 'Bangalore', 'Amazon', 'Laptop', 1, 65000, 18);
    """,

    """
    INSERT INTO invoices (invoice_date, location, vendor, item, quantity, amount, gst_rate)
    VALUES ('2025-01-05', 'Chennai', 'Flipkart', 'Headphones', 2, 4000, 18);
    """,

    """
    INSERT INTO invoices (invoice_date, location, vendor, item, quantity, amount, gst_rate)
    VALUES ('2025-01-10', 'Kochi', 'Reliance', 'Office Chair', 1, 9000, 12);
    """,

    """
    INSERT INTO invoices (invoice_date, location, vendor, item, quantity, amount, gst_rate)
    VALUES ('2025-01-12', 'Bangalore', 'Croma', 'Smartphone', 1, 30000, 18);
    """,

    """
    INSERT INTO invoices (invoice_date, location, vendor, item, quantity, amount, gst_rate)
    VALUES ('2025-01-15', 'Hyderabad', 'IKEA', 'Study Table', 1, 12000, 12);
    """,

    """
    INSERT INTO invoices (invoice_date, location, vendor, item, quantity, amount, gst_rate)
    VALUES ('2025-01-15', 'Mumbai', 'DMart', 'Groceries', 5, 3500, 5);
    """
]

for q in seed_queries:
    print(run_sql_tool(q))


Query executed successfully.
Query executed successfully.
Query executed successfully.
Query executed successfully.
Query executed successfully.
Query executed successfully.


In [13]:
print(run_sql_tool("SELECT * FROM invoices;"))


[{'id': 1, 'invoice_date': '2025-01-05', 'location': 'Bangalore', 'vendor': 'Amazon', 'item': 'Laptop', 'quantity': 1, 'amount': 65000.0, 'gst_rate': 18.0}, {'id': 2, 'invoice_date': '2025-01-10', 'location': 'Chennai', 'vendor': 'Flipkart', 'item': 'Headphones', 'quantity': 2, 'amount': 4000.0, 'gst_rate': 18.0}, {'id': 3, 'invoice_date': '2025-01-05', 'location': 'Bangalore', 'vendor': 'Amazon', 'item': 'Laptop', 'quantity': 1, 'amount': 65000.0, 'gst_rate': 18.0}, {'id': 4, 'invoice_date': '2025-01-05', 'location': 'Chennai', 'vendor': 'Flipkart', 'item': 'Headphones', 'quantity': 2, 'amount': 4000.0, 'gst_rate': 18.0}, {'id': 5, 'invoice_date': '2025-01-10', 'location': 'Kochi', 'vendor': 'Reliance', 'item': 'Office Chair', 'quantity': 1, 'amount': 9000.0, 'gst_rate': 12.0}, {'id': 6, 'invoice_date': '2025-01-12', 'location': 'Bangalore', 'vendor': 'Croma', 'item': 'Smartphone', 'quantity': 1, 'amount': 30000.0, 'gst_rate': 18.0}, {'id': 7, 'invoice_date': '2025-01-15', 'location':

In [14]:
print(run_sql_tool(
    "SELECT vendor, item, amount FROM invoices WHERE invoice_date = '2025-01-05';"
))


[{'vendor': 'Amazon', 'item': 'Laptop', 'amount': 65000.0}, {'vendor': 'Amazon', 'item': 'Laptop', 'amount': 65000.0}, {'vendor': 'Flipkart', 'item': 'Headphones', 'amount': 4000.0}]


In [15]:
print(run_sql_tool(
    "SELECT invoice_date, vendor, item FROM invoices WHERE location = 'Bangalore';"
))


[{'invoice_date': '2025-01-05', 'vendor': 'Amazon', 'item': 'Laptop'}, {'invoice_date': '2025-01-05', 'vendor': 'Amazon', 'item': 'Laptop'}, {'invoice_date': '2025-01-12', 'vendor': 'Croma', 'item': 'Smartphone'}]


In [16]:
print(run_sql_tool(
    "SELECT vendor, item, gst_rate FROM invoices WHERE gst_rate = 12;"
))


[{'vendor': 'Reliance', 'item': 'Office Chair', 'gst_rate': 12.0}, {'vendor': 'IKEA', 'item': 'Study Table', 'gst_rate': 12.0}]


In [2]:
import os
from dotenv import load_dotenv

from sentence_transformers import SentenceTransformer
import pinecone


In [3]:
load_dotenv()

PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
PINECONE_ENV = os.getenv("PINECONE_ENV")

assert PINECONE_API_KEY is not None, "PINECONE_API_KEY not set"
assert PINECONE_ENV is not None, "PINECONE_ENV not set"

print("Pinecone environment loaded ✅")


Pinecone environment loaded ✅


In [4]:
pc = pinecone.Pinecone(api_key=PINECONE_API_KEY)
print("Pinecone client initialized ✅")

Pinecone client initialized ✅


In [5]:
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")
EMBEDDING_DIM = embedding_model.get_sentence_embedding_dimension()

print("Embedding dimension:", EMBEDDING_DIM)

Loading weights:   0%|          | 0/103 [00:00<?, ?it/s]

BertModel LOAD REPORT from: sentence-transformers/all-MiniLM-L6-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


Embedding dimension: 384


In [7]:
from pinecone import ServerlessSpec

INDEX_NAME = "gst-rules-index"

existing_indexes = pc.list_indexes().names()

if INDEX_NAME not in existing_indexes:
    pc.create_index(
        name=INDEX_NAME,
        dimension=EMBEDDING_DIM,
        metric="cosine",
        spec=ServerlessSpec(
            cloud="aws",
            region="us-east-1"
        )
    )
    print("Pinecone index created ✅")
else:
    print("Pinecone index already exists ✅")


Pinecone index created ✅


In [8]:
index = pc.Index(INDEX_NAME)
print("Connected to Pinecone index ✅")

Connected to Pinecone index ✅


In [19]:
GST_DOC_PATH = "../data/gst_rules.txt"

with open(GST_DOC_PATH, "r", encoding="utf-8") as f:
    gst_text = f.read()

# Simple line-based chunking
gst_chunks = [line.strip() for line in gst_text.split("\n") if line.strip()]

print("GST chunks loaded:", len(gst_chunks))
gst_chunks


GST chunks loaded: 22


['Electronics such as laptops, smartphones, tablets, and accessories attract 18 percent GST.',
 'Large home appliances including refrigerators, washing machines, and air conditioners attract 18 percent GST.',
 'Furniture items such as tables, chairs, beds, and cupboards attract 12 percent GST.',
 'Office furniture and modular workstations attract 12 percent GST.',
 'Essential goods such as groceries, cereals, rice, wheat, milk, and vegetables attract 5 percent GST.',
 'Packaged food items including branded snacks and processed foods attract 12 percent GST.',
 'Luxury items such as premium cars, high-end motorcycles, and luxury watches attract 28 percent GST.',
 'Clothing priced below 1000 rupees per item attracts 5 percent GST.',
 'Clothing priced above 1000 rupees per item attracts 12 percent GST.',
 'Services such as consulting, professional services, IT services, and software services attract 18 percent GST.',
 'Hotel accommodation priced below 1000 rupees per day is exempt from GST

In [20]:
vectors = []

for i, chunk in enumerate(gst_chunks):
    embedding = embedding_model.encode(
        chunk, normalize_embeddings=True
    ).tolist()

    vectors.append(
        (
            f"gst-{i}",
            embedding,
            {"text": chunk}
        )
    )

index.upsert(vectors)
print("GST rules embedded and stored in Pinecone ✅")


GST rules embedded and stored in Pinecone ✅


In [21]:
query = "What is the GST rate for electronics?"

query_embedding = embedding_model.encode(
    query, normalize_embeddings=True
).tolist()

results = index.query(
    vector=query_embedding,
    top_k=3,
    include_metadata=True
)

for match in results["matches"]:
    print(match["metadata"]["text"])


Electronics such as laptops, smartphones, tablets, and accessories attract 18 percent GST.
Services such as consulting, professional services, IT services, and software services attract 18 percent GST.
Large home appliances including refrigerators, washing machines, and air conditioners attract 18 percent GST.
